# Finding Entities in Multiple Sclerosis Research

This isn't as much to develop more for Gregory, it's to see what I can do with Spacy and Named Entity Recognition (NER). This is a Jupyter notebook because I want to try giving it a proper use and because it will make it easier to achieve two goals.

1. Show others what my thought process was.
2. Make it easier to ask questions to people who know more than me.
3. Discover what is the best NER model to analyse Multiple Sclerosis (MS) articles

## Data sources

https://api.gregory-ms.com/articles/all

## Initilize modules and get data

In [17]:
%%capture

import os
import sys
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from pathlib import Path
!{sys.executable} -m pip install scispacy
import scispacy
!{sys.executable} -m pip install spacy
import spacy 
!{sys.executable} -m pip install pandas
import pandas as pd
!{sys.executable} -m pip install requests
import requests
from spacy import displacy

# NLP Models
!{sys.executable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_sm-0.5.0.tar.gz
!{sys.executable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_md-0.5.0.tar.gz
!{sys.executable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_scibert-0.5.0.tar.gz
!{sys.executable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_lg-0.5.0.tar.gz
!{sys.executable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_craft_md-0.5.0.tar.gz
!{sys.executable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_jnlpba_md-0.5.0.tar.gz
!{sys.executable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz
!{sys.executable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bionlp13cg_md-0.5.0.tar.gz


In [2]:

url = 'https://api.gregory-ms.com/articles/all'

urlData = requests.get(url).content

df = pd.read_json(urlData)

df

,article_id,title,summary,link,published_date,relevant,discovery_date,ml_prediction_gnb,ml_prediction_lr,noun_phrases,source,source__link,source__language,sources__subject
0,1138,The Relationship Between Walking Speed and the...,&lt;div&gt;&lt;p style&#x3D;&quot;color: #4aa5...,https://pubmed.ncbi.nlm.nih.gov/33847188/?utm_...,2021-01-06T00:00:00.000Z,NaN,2021-04-13T21:19:40.000Z,0.0,0.0,"[The Relationship, Walking Speed, the Energeti...",PubMed,https://pubmed.ncbi.nlm.nih.gov/rss/search/10g...,en,Multiple Sclerosis
1,1139,Microglial changes associated with meningeal i...,&lt;div&gt;&lt;p style&#x3D;&quot;color: #4aa5...,https://pubmed.ncbi.nlm.nih.gov/33846617/?utm_...,2021-01-05T00:00:00.000Z,NaN,2021-04-13T21:19:40.000Z,0.0,0.0,"[Microglial changes, meningeal inflammation, m...",PubMed,https://pubmed.ncbi.nlm.nih.gov/rss/search/10g...,en,Multiple Sclerosis
2,6317,Inadequate Vaccine Responses in Children With ...,&lt;div&gt;&lt;p style&#x3D;&quot;color: #4aa5...,https://pubmed.ncbi.nlm.nih.gov/34926358/?utm_...,2021-12-20T11:00:00.000Z,NaN,2021-12-20T19:13:06.000Z,0.0,0.0,"[Inadequate Vaccine Responses, Children, Multi...",PubMed,https://pubmed.ncbi.nlm.nih.gov/rss/search/10g...,en,Multiple Sclerosis
3,6324,Recent advances in clinical trials targeting t...,&lt;div&gt;&lt;p style&#x3D;&quot;color: #4aa5...,https://pubmed.ncbi.nlm.nih.gov/34929198/?utm_...,2021-12-20T11:00:00.000Z,NaN,2021-12-21T07:13:06.000Z,0.0,0.0,"[Recent advances, clinical trials, the kynuren...",PubMed,https://pubmed.ncbi.nlm.nih.gov/rss/search/10g...,en,Multiple Sclerosis
4,6327,TDP-43 Pathology in Alzheimer&#39;s Disease,&lt;div&gt;&lt;p style&#x3D;&quot;color: #4aa5...,https://pubmed.ncbi.nlm.nih.gov/34930382/?utm_...,2021-12-21T11:00:00.000Z,NaN,2021-12-21T13:13:06.000Z,0.0,0.0,"[TDP-43 Pathology, Alzheimer&#39;s Disease]",PubMed,https://pubmed.ncbi.nlm.nih.gov/rss/search/10g...,en,Multiple Sclerosis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7988,12696,Does the Serum Expression Level of High-Mobili...,Background: There is a strong need to identify...,https://pubmed.ncbi.nlm.nih.gov/35263889/?utm_...,2022-03-10T11:00:00.000Z,NaN,2022-03-10T12:42:30.855Z,0.0,0.0,"[the Serum Expression Level, High-Mobility Gro...",PubMed,https://pubmed.ncbi.nlm.nih.gov/rss/search/10g...,en,Multiple Sclerosis
7989,14071,The microbiota restrains neurodegenerative mic...,Background: The gut microbiota can affect neur...,https://microbiomejournal.biomedcentral.com/ar...,2022-03-11T00:00:00.000Z,NaN,2022-03-11T08:17:48.236Z,0.0,0.0,"[The microbiota, a model, amyotrophic lateral ...",BioMedCentral,https://www.biomedcentral.com/search?searchTyp...,en,Multiple Sclerosis
7990,14074,Autologous treatment for ALS with implication ...,Background: Amyotrophic lateral sclerosis (ALS...,https://translationalneurodegeneration.biomedc...,2022-03-11T00:00:00.000Z,NaN,2022-03-11T08:17:48.488Z,0.0,0.0,"[Autologous treatment, ALS, implication, broad...",BioMedCentral,https://www.biomedcentral.com/search?searchTyp...,en,Multiple Sclerosis
7991,14538,Timed Up &amp; Go (TUG) With Cognitive and Man...,Timed Up &amp; Go (TUG) With Cognitive and Man...,https://www.apta.org/patient-care/evidence-bas...,2022-03-11T14:20:18.802Z,NaN,2022-03-11T14:17:49.099Z,0.0,0.0,"[Timed, Go, TUG, Cognitive and Manual Tasks, M...",APTA,https://www.apta.org/search?Q=%22Multiple+Scle...,en,Multiple Sclerosis


In [3]:
summary = df.loc[4201, 'summary']
summary

'&lt;div&gt;&lt;p style&#x3D;&quot;color: #4aa564;&quot;&gt;Eur J Neurol. 2021 Jun 21. doi: 10.1111&#x2F;ene.14990. Online ahead of print.&lt;&#x2F;p&gt;&lt;p&gt;&lt;b&gt;ABSTRACT&lt;&#x2F;b&gt;&lt;&#x2F;p&gt;&lt;p xmlns:xlink&#x3D;&quot;http:&#x2F;&#x2F;www.w3.org&#x2F;1999&#x2F;xlink&quot; xmlns:mml&#x3D;&quot;http:&#x2F;&#x2F;www.w3.org&#x2F;1998&#x2F;Math&#x2F;MathML&quot; xmlns:p1&#x3D;&quot;http:&#x2F;&#x2F;pubmed.gov&#x2F;pub-one&quot;&gt;BACKGROUND: Limited information is available on incidence and outcomes of COVID-19 in patients with multiple sclerosis (MS). This study investigated the risks of SARS-CoV-2 infection and COVID-19-related outcomes in patients with MS, and compared these with the general population.&lt;&#x2F;p&gt;&lt;p xmlns:xlink&#x3D;&quot;http:&#x2F;&#x2F;www.w3.org&#x2F;1999&#x2F;xlink&quot; xmlns:mml&#x3D;&quot;http:&#x2F;&#x2F;www.w3.org&#x2F;1998&#x2F;Math&#x2F;MathML&quot; xmlns:p1&#x3D;&quot;http:&#x2F;&#x2F;pubmed.gov&#x2F;pub-one&quot;&gt;METHODS: A re

Summary includes html, so we need to clean the data

In [4]:
import html
summary = html.unescape(summary)

from bs4 import BeautifulSoup
soup = BeautifulSoup(summary, features="html.parser")
for script in soup(["script", "style"]):
    script.extract()    # rip it out
summary = soup.get_text()
summary 



'Eur J Neurol. 2021 Jun 21. doi: 10.1111/ene.14990. Online ahead of print.ABSTRACTBACKGROUND: Limited information is available on incidence and outcomes of COVID-19 in patients with multiple sclerosis (MS). This study investigated the risks of SARS-CoV-2 infection and COVID-19-related outcomes in patients with MS, and compared these with the general population.METHODS: A regional registry was created to collect data on incidence, hospitalization rates, intensive care unit (ICU) admission and death in patients with MS and COVID-19. National government outcomes and seroprevalence data were used for comparison. The study was conducted at 14 specialist MS treatment centers in Madrid, Spain, between February and May 2020.RESULTS: Two-hundred nineteen patients were included in the registry, 51 of whom were hospitalized with COVID-19. The mean age ± standard deviation was 45.3 ± 12.4 years and the mean duration of MS was 11.9 ± 8.9 years. The infection incidence rate was lower in patients wit

Let's look at the output of 'en_core_sci_md' as a NER, and we'll see that it identifies entities, but does not show what they are.

In [5]:
nlp = spacy.load('en_core_sci_md')
doc = nlp(summary)
displacy_image = displacy.render(doc,  style = 'ent' )
displacy_image

Same thing, this time with `en_ner_jnlpba_md` as a NER model, and we don't see any entities at all.

There are four models available for NER on science articles that we are going to test. These all come from  [SciSpacy](https://allenai.github.io/scispacy/).	


| **Model**            | **Description**                                                                                                      | **Install URL** | **Tested** |
|----------------------|----------------------------------------------------------------------------------------------------------------------|-----------------|---|
| en_core_sci_sm       | A full spaCy pipeline for biomedical data.                                                                           | [Download](https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_sm-0.5.0.tar.gz)        | No, we tested en_core_sci_md because it has a larger corpus |
| en_core_sci_md       | A full spaCy pipeline for biomedical data with a larger vocabulary and 50k word vectors.                             | [Download](https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_md-0.5.0.tar.gz)        | Yes |
| en_core_sci_scibert  | A full spaCy pipeline for biomedical data with a ~785k vocabulary and allenai/scibert-base as the transformer model. | [Download](https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_scibert-0.5.0.tar.gz)        | Yes |
| en_core_sci_lg       | A full spaCy pipeline for biomedical data with a larger vocabulary and 600k word vectors.                            | [Download](https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_lg-0.5.0.tar.gz)        | Yes |
| en_ner_craft_md      | A spaCy NER model trained on the CRAFT corpus.                                                                       | [Download](https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_craft_md-0.5.0.tar.gz)        | Yes |
| en_ner_jnlpba_md     | A spaCy NER model trained on the JNLPBA corpus.                                                                      | [Download](https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_jnlpba_md-0.5.0.tar.gz)        | Yes | 
| en_ner_bc5cdr_md     | A spaCy NER model trained on the BC5CDR corpus.                                                                      | [Download](https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz)        | Yes |
| en_ner_bionlp13cg_md | A spaCy NER model trained on the BIONLP13CG corpus.                                                                  | [Download](https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bionlp13cg_md-0.5.0.tar.gz)        | Yes |



In [6]:
nlp_sci = spacy.load('en_core_sci_md')
nlp_scibert = spacy.load('en_core_sci_scibert')
nlp_core_sci_lg = spacy.load('en_core_sci_lg')
nlp_cr = spacy.load('en_ner_craft_md')
nlp_bc = spacy.load('en_ner_bc5cdr_md')
nlp_bi = spacy.load('en_ner_bionlp13cg_md')
nlp_jn = spacy.load('en_ner_jnlpba_md')


In [7]:
doc = nlp_sci(summary)
displacy_image = displacy.render(doc, style = 'ent',jupyter=True)

In [8]:
doc = nlp_scibert(summary)
displacy_image = displacy.render(doc, style = 'ent',jupyter=True)

/Users/brunoamaral/Labs/gregory/env/lib/python3.7/site-packages/torch/cuda/amp/autocast_mode.py:120: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


In [9]:
doc = nlp_core_sci_lg(summary)
displacy_image = displacy.render(doc, style = 'ent',jupyter=True)

In [10]:
doc = nlp_cr(summary)
displacy_image = displacy.render(doc, style = 'ent',jupyter=True)

/Users/brunoamaral/Labs/gregory/env/lib/python3.7/site-packages/spacy/displacy/__init__.py:200: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [11]:
doc = nlp_bc(summary)
displacy_image = displacy.render(doc, style = 'ent',jupyter=True)

In [12]:
doc = nlp_bi(summary)
displacy_image = displacy.render(doc, style = 'ent',jupyter=True)

In [13]:
doc = nlp_jn(summary)
displacy_image = displacy.render(doc, style = 'ent',jupyter=True)

# Conclusion so far

We are able to identify some proteins, aparently, but not sure if they are relevant for research.